In [85]:
# We're going to install this module to help us parse datetimes from the raw dataset
!pip install dateparser

In [105]:
from pymongo import MongoClient, InsertOne, UpdateOne
import pprint
import dateparser
from bson.json_util import loads

In [87]:
client = MongoClient("mongodb://localhost:27017/introToMongoDB?readPreference=primary&appname=MongoDB%20Compass&ssl=false")
people_raw = client["introToMongoDB"]["people-raw"]

In [106]:
# Confirm that 50,474 documents are in your collection before moving on
people_raw.count_documents({})

50474

In [107]:
people_with_string_birthdays = people_raw.find ({"birthday": {"$type" : "string"}})

people_with_string_birthdays.count()

<ipython-input-107-08e3c59cbbd3>:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  people_with_string_birthdays.count()


10382

In [108]:
batch_size = 1000
inserts = []
count = 0
updates = []
# Again, we're updating several thousand documents, so this will take a little while
for person in people_with_string_birthdays:
    # Pymongo converts datetime objects into BSON Dates. The dateparser.parse function returns a
    # datetime object, so we can simply do the following to update the field properly.
    # Replace ZZZZ with the correct update operator
    updates.append(UpdateOne({ "_id": person["_id"] }, { "$set": { "birthday": dateparser.parse(person["birthday"]) } }))
    
    count += 1
                       
    if count == batch_size:
        people_raw.bulk_write(updates)
        updates = []
        count = 0
        
if updates:         
    people_raw.bulk_write(updates)
    count = 0

In [111]:
# # If everything went well this should be zero
# people_with_string_birthdays = people_raw.aggregate([
#     {"$match": {"birthday": {"$type" : "string"}}},
#     {"$group": { "_id": None, "count": { "$sum": 1 }}}
# ])

# for document in people_with_string_birthdays:
#     print(document)

In [112]:
people_with_string_birthdays = people_raw.find ({"birthday": {"$type" : "string"}})

people_with_string_birthdays.count()

<ipython-input-112-08e3c59cbbd3>:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  people_with_string_birthdays.count()


0